In [158]:
from typing import TypedDict, Annotated
from langgraph.graph import START,END,StateGraph
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

load_dotenv()

True

In [159]:
gpt_llm= ChatOpenAI(model="gpt-4o-mini")

In [160]:
class Conversation(TypedDict):
    messages: Annotated[list[BaseMessage],add_messages]


In [161]:
def chatbot(state:Conversation)-> Conversation:
    prompt=f"You are an AI assistant that helps answering questions. Please help in answer the question: \n {state['messages']}"
    messages=gpt_llm.invoke(prompt)
    return {"messages":[messages]}

In [162]:
checkpointer= MemorySaver()
graph = StateGraph(Conversation)
graph.add_node("chatbot",chatbot)
graph.add_edge(START,"chatbot")
graph.add_edge("chatbot",END)



In [ ]:
workflow=graph.compile(checkpointer=checkpointer)

In [164]:
thread_id= "1234"
while True:
    user_input= input("Your query:")
    print(f"User:{user_input}")
    if user_input.strip().lower() in ["exit","bye","quit"]:
        break

    config={"configurable":{"thread_id":thread_id}}
    response=workflow.invoke({"messages":user_input},config=config)
    print("AI:", response['messages'][-1].content)


User:Hello, My name is Gaurav
AI: Hello Gaurav! How can I assist you today?
User:who am I?
AI: You are Gaurav, a person who has introduced yourself at the beginning of the conversation. How can I assist you further today?
User:bye
